In [1]:
import json
import copy
from geopy import distance
from tqdm import tqdm
from rtree import index
from multiprocessing import Pool, cpu_count

# Ubicación de los archivos
file1 = './geojsons/TORRES_FINAL_PRUEBA.geojson'
file2 = './geojsons/TORRES_500KV_19S_PRUEBA_CODIGO.geojson'  # Reemplaza con la ruta a tu segundo archivo

# Abre y lee los archivos GeoJSON
with open(file1) as f:
    data1 = json.load(f)
with open(file2) as f:
    data2 = json.load(f)

# Inicializa los contadores
contador_elementos = 0
contador_propiedades = 0

# Crea una copia profunda de data1
data1_copy = copy.deepcopy(data1)

# Crear un índice espacial para data2
idx = index.Index()
feature_dict = {}
for pos, feature in enumerate(data2['features']):
    coords = feature['geometry']['coordinates']
    feature_dict[pos] = feature
    idx.insert(pos, (*coords, *coords))

# Función para procesar cada feature1
def process_feature(feature1):
    global contador_elementos, contador_propiedades

    tension = feature1['properties'].get('Tensión')
    if tension != '500kV':
        return feature1, 0, 0
    
    f1_coords = feature1['geometry']['coordinates']
    
    # Encontrar los elementos cercanos usando el índice espacial
    nearby_elements = list(idx.nearest((*f1_coords, *f1_coords), 10))
    
    for pos in nearby_elements:
        feature2 = feature_dict[pos]
        f2_coords = feature2['geometry']['coordinates']
        
        dist = distance.distance(f1_coords, f2_coords).m
        
        if dist < 15:
            # Guarda la propiedad "Tensión" de feature1
            tension = feature1['properties'].get('Tensión')
            # Actualiza las propiedades
            feature1['properties'] = feature2['properties']
            # Restaura la propiedad "Tensión" de feature1
            if tension is not None:
                feature1['properties']['Tensión'] = tension
            # Incrementa los contadores
            return feature1, 1, len(feature2['properties'])
    return feature1, 0, 0

# Usar multiprocessing para paralelizar el procesamiento
with Pool(cpu_count()) as pool:
    results = list(tqdm(pool.imap(process_feature, data1_copy['features']), total=len(data1_copy['features'])))

# Actualizar data1_copy y los contadores
for i, (updated_feature, elem_count, prop_count) in enumerate(results):
    data1_copy['features'][i] = updated_feature
    contador_elementos += elem_count
    contador_propiedades += prop_count

# Guarda la copia de data1 con las propiedades actualizadas
output_file = './geojsons/TORRES_FINAL_PRUEBA_ACTUALIZADO.geojson'  # Reemplaza con la ruta a tu archivo de salida
with open(output_file, 'w') as f:
    json.dump(data1_copy, f, ensure_ascii=False, indent=4)

# Imprime los contadores
print(f'Se encontraron {contador_elementos} elementos con las mismas coordenadas.')
print(f'Se actualizaron {contador_propiedades} propiedades.')


100%|██████████| 39193/39193 [00:02<00:00, 17857.51it/s]


Se encontraron 3777 elementos con las mismas coordenadas.
Se actualizaron 22662 propiedades.
